In [1]:
!pip install wrds

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [2]:
import wrds

# Connect to WRDS
db = wrds.Connection()

Enter your WRDS username [root]:dreamspartan
Enter your password:··········


OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: FATAL:  PAM authentication failed for user "dreamspartan"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
!echo $HOME
!ls -la /root/.pgpass
!chmod 0600 ~/.pgpass


In [ ]:
db.create_pgpass_file()

In [ ]:
tables = db.list_tables(library='optionm')  # Assuming 'optionm' is the schema for OptionMetrics
print(tables)

In [ ]:
db.describe_table(library='optionm', table='opprcd2020') # Added the library argument


In [ ]:
from datetime import datetime, timedelta
import pandas as pd

# Define the start and end dates for the query
year = '2023'
table = 'opprcd'+ year
start_date = year + '-07-10'
end_date = year + '-07-12'

# exdate = end_date + timedelta(days=30)

# Define the SQL query to retrieve S&P 500 options data (assuming S&P 500 index ticker is 'SPX')

query = f"""
    SELECT date, exdate, cp_flag, strike_price, best_bid, best_offer,
          impl_volatility, delta, gamma, vega, theta, optionid
    FROM optionm.{table}
    WHERE secid = '108105' -- This is the secid for SPX index options
    AND date BETWEEN '{start_date}' AND '{end_date}'  -- Filter by date range
    AND exdate < '{end_date}'::date + interval '30 days'  -- 1 month expiry
    ORDER BY date DESC  -- Order by date to get recent data first
    LIMIT 100;  -- Limit results for testing
"""

data = db.raw_sql(query)

# Display the first few rows of the fetched data
print(data.head())

# Close the WRDS connection
# db.close()


In [ ]:
data.describe()
data.head()

In [1]:
!pip install sec-cik-mapper

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
from sec_cik_mapper import StockMapper

# Initialize a stock mapper instance
mapper = StockMapper()

# Get mapping from CIK to tickers
cik_to_tickers = mapper.cik_to_tickers

# Get mapping from ticker to CIK
ticker_to_cik = mapper.ticker_to_cik

# Print some examples
print("CIK to Tickers:")
for cik, tickers in list(cik_to_tickers.items())[:5]:
    print(f"CIK: {cik}, Tickers: {tickers}")

print("\nTicker to CIK:")
for ticker, cik in list(ticker_to_cik.items())[:5]:
    print(f"Ticker: {ticker}, CIK: {cik}")

CIK to Tickers:
CIK: 0000001750, Tickers: {'AIR'}
CIK: 0000001800, Tickers: {'ABT'}
CIK: 0000001961, Tickers: {'WDDD'}
CIK: 0000002098, Tickers: {'ACU'}
CIK: 0000002178, Tickers: {'AE'}

Ticker to CIK:
Ticker: AIR, CIK: 0000001750
Ticker: ABT, CIK: 0000001800
Ticker: WDDD, CIK: 0000001961
Ticker: ACU, CIK: 0000002098
Ticker: AE, CIK: 0000002178


In [6]:
from sec_cik_mapper import StockMapper
import openpyxl

# Initialize a stock mapper instance
mapper = StockMapper()

# Load the Excel workbook
workbook_path = r"/Users/mukeshwaranbaskaran/Downloads/Complete-List-of-Biotech-Stocks-Listed-on-NASDAQ-Jan-1-24.xlsx"
workbook = openpyxl.load_workbook(workbook_path)

# Select the active sheet
sheet = workbook.active

# Specify the column index where the tickers are located (0 for 'A', 1 for 'B', etc.)
ticker_column_index = 2 

# Get the tickers from the Excel column
tickers = [cell.value for cell in sheet.iter_rows(min_row=2, min_col=ticker_column_index + 1, max_col=ticker_column_index + 1, values_only=True) if cell[0]]

# Create a dictionary to store the mappings
mapping = {}

# Iterate over the tickers and get the corresponding CIKs
for ticker in tickers:
    cik = mapper.ticker_to_cik.get(ticker)
    if cik:
        mapping[ticker] = cik
    else:
        print(f"No CIK found for ticker: {ticker}")

# Print the mapping
for ticker, cik in mapping.items():
    print(f"Ticker: {ticker}, CIK: {cik}")

# Optionally, write the mapping back to a new sheet in the same workbook
output_sheet = workbook.create_sheet(title='Ticker to CIK Mapping')

# Write header
output_sheet.append(['Ticker', 'CIK'])

# Write mappings
for ticker, cik in mapping.items():
    output_sheet.append([ticker, cik])

# Save the workbook with the new sheet
output_workbook_path = r"/Users/mukeshwaranbaskaran/Downloads/Biotech_Stocks_Mapping.xlsx"
workbook.save(output_workbook_path)

print(f"Mapping saved to {output_workbook_path}")


AttributeError: 'tuple' object has no attribute 'value'